This notebook fits the baseline model. It predicts on the test data without doing any feature engineering, using the xgboost library.

In [1]:
import time
import pandas as pd
import numpy as np
import wandb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.metrics import accuracy_score

df_train = pd.read_csv('data/train.csv').drop(['id'], axis=1)
df_train['source'] = 'simulation'

df_test = pd.read_csv('data/test.csv')
test_ids = df_test.id
dt_test = df_test.drop(['id'], axis=1)

df_supp = pd.read_csv('data/cirrhosis.csv').drop(['ID'], axis=1)
df_supp['source'] = 'original'

# merge supplemental data
df_train = pd.concat([df_train, df_supp]).reset_index(drop=True)
train_target = df_train['Status']

TARGET = 'Status'
SKEWED_FEATS = ['Bilirubin', 'Cholesterol', 'Copper', 'Prothrombin', 'Alk_Phos']
CAT_FEATS = ['Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Stage']
NUM_FEATS = [x for x in df_train.columns if x not in CAT_FEATS and x != TARGET]
NUM_FEATS.remove('source')
ORIG_FEATS = df_train.drop(TARGET, axis=1).columns.tolist()


# Prep Data for XGBoost

## Helper Functions

In [2]:
def log_transformation(data, skewed_features):
    df_log_feats = pd.DataFrame()
    for col in skewed_features:
        name = f'{col}_log'
        df_log_feats[name] = np.log(data[col])
    return df_log_feats

def encode(data):
    X_raw = data.iloc[:, 0:18]

    # encode the categorical features
    X_cat = X_raw[CAT_FEATS]
    X_num = np.array(X_raw[NUM_FEATS])
    oe = OrdinalEncoder()
    X_cat = oe.fit_transform(X_cat)
    X_cat = pd.DataFrame(X_cat, columns=CAT_FEATS)
    X_num = pd.DataFrame(X_num, columns=NUM_FEATS)
    return X_cat, X_num

## Encoding

In [3]:
# create features and target arrays
X, y = df_train.drop(TARGET, axis=1), df_train[[TARGET]]

# encode features
X_cat, X_num = encode(X)
X = pd.concat([X_cat, X_num], axis=1)

# encode target
le = LabelEncoder()
y = le.fit_transform(np.ravel(y))

## Transformations

In [4]:
# n symptoms
# recode edema "S" and "Y" to both indicate edema for the purpose of this count variable
X['Edema'] = np.where(X['Edema'] == 2, 1, X['Edema'])
X['N_Symptoms'] = X.Edema + X.Spiders + X.Ascites + X.Hepatomegaly

# logarithmic transformations
def get_logs(data, skewed_features):
    X_log = X[skewed_features]
    other_feats = data.columns.difference(skewed_features).tolist()
    X_other = X[other_feats]
    X_log = log_transformation(X_log, skewed_features)
    return pd.concat([X_log, X_other], axis=1)

X = get_logs(X, SKEWED_FEATS)

# Age at Diagnosis
X['Dgns_Age'] = X['Age'] - X['N_Days']

## Train/Test Split

In [5]:
# do train/test split on the data
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=.2, random_state=1)

# Fit Model

## Define wandb sweep parameters

In [6]:
sweep_config = {
    "method": "random", # sweep method
    "metric": {
        "name": "accuracy",
        "goal": "maximize"
    },
    "parameters": {
        "booster": { # tree-based or linear functions?
            "values": ["gbtree", "gblinear"]
        },
        "max_depth": { # depth of individual trees
            "values": [3, 6, 9, 12]
        },
        "learning_rate": {
            "values": [0.1, 0.05, 0.2]
        },
        "subsample": { # the proportion of instances to take a sub-sample per iteration
            "values": [1, 0.5, 0.3] 
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project='cirrhosis-xgb-sweeps')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: jbbh56vz
Sweep URL: https://wandb.ai/reeds-team/cirrhosis-xgb-sweeps/sweeps/jbbh56vz


In [7]:
# define training function
def train():
    config_defaults = {
        "booster": "gbtree",
        "max_depth": 3,
        "learning_rate": 0.1,
        "subsample": 1,
        "seed": 1,
    }

    wandb.init(config=config_defaults) # set defaults. They'll be over-ridden later.
    config = wandb.config

    model = XGBClassifier(
        booster=config.booster,
        max_depth=config.max_depth,
        learning_rate=config.learning_rate,
        subsample=config.subsample,
    )
    
    model.fit(X_train, y_train)

    # get predictions on dev set
    y_pred = model.predict(X_dev)
    predictions = [round(value) for value in y_pred]

    # evaluate predictions
    accuracy = accuracy_score(y_dev, predictions)
    print(f"Accuracy: {accuracy:.0%}")
    wandb.log({"accuracy": accuracy})

    model.save_model(f'checkpoints/{config.booster}"-"{config.max_depth}"-"{config.learning_rate}"-"{config.subsample}"-"{time.time()}.json') 
    # with a categorical target, if its not json then it throws an error

In [ ]:
wandb.login()

# specify the sweep, the training function, and the number of sweeps
wandb.agent(sweep_id, train, count=25)

wandb: Currently logged in as: reedmerrill (reeds-team). Use `wandb login --relogin` to force relogin
wandb: Agent Starting Run: nwa0zpi3 with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.05
wandb: 	max_depth: 12
wandb: 	subsample: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Accuracy: 83%


accuracy,▁
accuracy,0.82643


wandb: Agent Starting Run: zl8h7um9 with config:
wandb: 	booster: gbtree
wandb: 	learning_rate: 0.1
wandb: 	max_depth: 3
wandb: 	subsample: 0.5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Ctrl + C detected. Stopping sweep.


Accuracy: 82%


# to-do

1. make early stopping and n_estimators work
    1. needs so refactoring between which args go in the constructor vs. the train method.
2. cross-validation